In [35]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [37]:
tf.keras.backend.clear_session()

In [38]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [39]:
data = pd.read_csv('2003_mo_model_input_ES.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value,Espírito Santo - Consumo de Cimento (t)
0,2003-1,8.177444,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,0.760059,...,19.768291,76.995517,2.181755,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,98.171240,0.268445,48.426
1,2003-2,8.171622,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,0.760222,...,19.767196,76.994127,2.180648,6.350834e+07,3.527412e+06,16.807398,5.951949e+07,97.897317,0.270512,54.969
2,2003-3,8.165799,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,0.760385,...,19.766557,76.993316,2.180003,6.359131e+07,3.530838e+06,16.815488,5.955649e+07,98.070539,0.271941,53.847
3,2003-4,8.159977,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,0.760548,...,19.766489,76.993229,2.179934,6.367428e+07,3.534263e+06,16.823577,5.959349e+07,98.460789,0.273321,50.375
4,2003-5,8.154154,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,0.760710,...,19.846793,77.041977,2.180096,6.375725e+07,3.537689e+06,16.831667,5.963048e+07,99.084134,0.274453,52.576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.800200,73.732416,1.592071,NaN,NaN,NaN,NaN,66.919726,0.505935,100.170
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.446098,74.981860,1.565408,NaN,NaN,NaN,NaN,66.756019,0.504835,85.832
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.911870,75.700549,1.570588,NaN,NaN,NaN,NaN,66.684567,0.502971,91.318
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.011517,76.180398,1.554322,NaN,NaN,NaN,NaN,66.664258,0.500670,77.187


In [40]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
0,-0.758586,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-0.557551,...,-0.342027,-0.316813,-0.005104,1.311648,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991
1,-0.763437,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-0.556231,...,-0.342437,-0.317363,-0.005801,1.306375,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951
2,-0.768288,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-0.555461,...,-0.342676,-0.317684,-0.006207,1.303299,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715
3,-0.773139,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-0.555379,...,-0.342702,-0.317718,-0.006251,1.302969,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649
4,-0.777990,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-0.511712,...,-0.304325,-0.277397,0.018188,1.303740,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.925704,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,1.304933,-0.798989,...,-0.891805,-0.627674,0.720000,-1.675993,1.073521,-0.934891,-1.155565,-0.875236,-1.727603,1.093145
188,0.920658,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,1.292490,-0.533704,...,-0.566031,-0.420484,0.345125,-1.306486,1.056445,-0.903403,-1.167749,-0.908951,-1.753915,1.108075
189,0.915611,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,1.280047,-0.593718,...,-0.168555,-0.168242,-0.055131,-0.942080,1.039370,-0.871915,-1.179933,-0.942665,-1.778143,1.130399
190,0.910564,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,1.267603,0.051244,...,0.244201,0.109748,-0.398952,-0.887727,1.022295,-0.840427,-1.192116,-0.976380,-1.806324,1.153917


In [41]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      48.646
1      43.597
2      56.331
3      48.488
4      52.856
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Espírito Santo - Consumo de Cimento (t), Length: 240, dtype: float64

In [42]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
0,-0.758586,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-0.557551,...,-0.342027,-0.316813,-0.005104,1.311648,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991
1,-0.763437,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-0.556231,...,-0.342437,-0.317363,-0.005801,1.306375,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951
2,-0.768288,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-0.555461,...,-0.342676,-0.317684,-0.006207,1.303299,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715
3,-0.773139,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-0.555379,...,-0.342702,-0.317718,-0.006251,1.302969,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649
4,-0.777990,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-0.511712,...,-0.304325,-0.277397,0.018188,1.303740,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.538868,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,1.408727,-0.094658,...,0.832716,0.238230,-2.008302,-0.345405,1.007109,-1.505857,-1.639139,-1.626063,-0.138284,1.037783
158,1.502037,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,1.408716,-0.166709,...,0.339955,-0.025443,-1.246005,-1.017572,1.017654,-1.496536,-1.616758,-1.583348,-0.199466,1.039895
159,1.465205,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,1.408704,-0.481831,...,-0.168876,-0.410117,-0.911422,-1.426643,1.028199,-1.487214,-1.594377,-1.540633,-0.263743,1.042387
160,1.428374,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,1.408692,-0.472102,...,-0.628926,-0.789563,-0.790466,-1.529834,1.038744,-1.477892,-1.571996,-1.497917,-0.329315,1.044800


In [43]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       48.646
1       43.597
2       56.331
3       48.488
4       52.856
        ...   
157     77.958
158    105.434
159     86.699
160     90.574
161     85.148
Name: Espírito Santo - Consumo de Cimento (t), Length: 162, dtype: float64

In [44]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [45]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [46]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
126,0.929165,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,0.503951,-1.289052,...,-1.216628,-1.023939,0.293795,-0.843046,0.864276,-0.433841,-0.655804,0.094697,1.142222,0.875311
127,0.963455,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,0.514753,-0.892232,...,-1.102395,-0.942981,0.218883,-0.499254,0.876404,-0.493112,-0.692821,0.038864,1.099243,0.884376
128,0.997745,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,0.525554,-0.363402,...,-0.835209,-0.619239,0.603559,-0.293131,0.888532,-0.552384,-0.729838,-0.016968,1.049585,0.894352
129,1.032035,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,0.536356,0.257851,...,-0.498985,-0.345165,0.527643,-0.101504,0.900660,-0.611655,-0.766855,-0.072801,1.007713,0.904024
130,1.066325,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,0.547157,0.470435,...,-0.153741,-0.056033,0.522163,-0.159016,0.912788,-0.670927,-0.803872,-0.128633,0.957549,0.912127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.925704,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,1.304933,-0.798989,...,-0.891805,-0.627674,0.720000,-1.675993,1.073521,-0.934891,-1.155565,-0.875236,-1.727603,1.093145
188,0.920658,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,1.292490,-0.533704,...,-0.566031,-0.420484,0.345125,-1.306486,1.056445,-0.903403,-1.167749,-0.908951,-1.753915,1.108075
189,0.915611,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,1.280047,-0.593718,...,-0.168555,-0.168242,-0.055131,-0.942080,1.039370,-0.871915,-1.179933,-0.942665,-1.778143,1.130399
190,0.910564,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,1.267603,0.051244,...,0.244201,0.109748,-0.398952,-0.887727,1.022295,-0.840427,-1.192116,-0.976380,-1.806324,1.153917


In [47]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [48]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [49]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
#                              return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.LSTM(36,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')     
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [50]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_val_loss = min(history.history['val_loss'][500:])
        current_loss = history.history['loss'][history.history['val_loss'].index(current_val_loss)]
        decision_value = (current_val_loss*current_val_loss)*current_loss 
        print(f"val_loss: {current_val_loss}; loss={current_loss} => decision_value={decision_value}")

        if decision_value < best_loss:
            best_loss = decision_value
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [51]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1014175174, 559912901, 3738782867, 3164524623, 459678084, 521972957, 3364902095, 1956626224, 1102861442, 2273285192, 2216637266, 1004041981, 1060018234, 2435205637, 946514342, 3179935386, 2473395117, 2616653252, 3458087772, 186818175]


Step: 0 ___________________________________________
val_loss: 92.90998840332031; loss=42.42375946044922 => decision_value=366213.17405376676
winner_seed: 1014175174


Step: 1 ___________________________________________
val_loss: 94.70541381835938; loss=38.85755920410156 => decision_value=348517.93291675334
winner_seed: 559912901


Step: 2 ___________________________________________
val_loss: 97.6640396118164; loss=27.91798210144043 => decision_value=266289.10131122824
winner_seed: 3738782867


Step: 3 ___________________________________________


2023-10-20 13:55:03.310919: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-20 13:55:03.344234: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-20 13:55:03.750286: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 93.61386108398438; loss=31.09452247619629 => decision_value=272498.5575162557


Step: 4 ___________________________________________
val_loss: 92.18518829345703; loss=31.845359802246094 => decision_value=270625.33685528


Step: 5 ___________________________________________
val_loss: 100.41798400878906; loss=44.703948974609375 => decision_value=450784.40716147603


Step: 6 ___________________________________________
val_loss: 95.28841400146484; loss=35.29991149902344 => decision_value=320519.02547647385


Step: 7 ___________________________________________
val_loss: 92.18196105957031; loss=38.11681365966797 => decision_value=323898.1556039195


Step: 8 ___________________________________________
val_loss: 94.169921875; loss=35.40810775756836 => decision_value=313998.1855672257


Step: 9 ___________________________________________
val_loss: 89.78982543945312; loss=28.450740814208984 => decision_value=229375.9254088932
winner_seed: 2273285192


Step: 10 __________________________

In [52]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 1s 77ms/step - loss: 11038.5537 - val_loss: 11096.4863
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 10895.2646 - val_loss: 10935.5029
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 10726.8320 - val_loss: 10743.1807
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 10526.6006 - val_loss: 10520.1768
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 10307.6484 - val_loss: 10290.9023
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 10079.4863 - val_loss: 10055.2988
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 9846.4541 - val_loss: 9813.3516
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 9606.2266 - val_loss: 9577.4590
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 9377.1641 - val_loss: 9350.0273
Epoch 10/10000
4/4 [======================

4/4 [==============================] - 0s 9ms/step - loss: 5148.8604 - val_loss: 5245.6387
Epoch 79/10000
4/4 [==============================] - 0s 9ms/step - loss: 5110.7651 - val_loss: 5207.0981
Epoch 80/10000
4/4 [==============================] - 0s 10ms/step - loss: 5072.2544 - val_loss: 5169.0225
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 5034.8008 - val_loss: 5131.0913
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 4996.9463 - val_loss: 5093.4629
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 4959.9648 - val_loss: 5055.8579
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 4922.7036 - val_loss: 5018.5547
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 4885.9106 - val_loss: 4981.5928
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 4849.1196 - val_loss: 4944.9746
Epoch 87/10000
4/4 [==============================] - 0s 9m

4/4 [==============================] - 0s 10ms/step - loss: 2845.7053 - val_loss: 2931.2039
Epoch 155/10000
4/4 [==============================] - 0s 9ms/step - loss: 2822.2412 - val_loss: 2907.8779
Epoch 156/10000
4/4 [==============================] - 0s 10ms/step - loss: 2799.2183 - val_loss: 2884.6394
Epoch 157/10000
4/4 [==============================] - 0s 11ms/step - loss: 2776.2891 - val_loss: 2861.5088
Epoch 158/10000
4/4 [==============================] - 0s 9ms/step - loss: 2753.4753 - val_loss: 2838.5493
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 2730.8198 - val_loss: 2815.8330
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 2708.4314 - val_loss: 2793.1833
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 2686.2366 - val_loss: 2770.6523
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 2663.9966 - val_loss: 2748.3679
Epoch 163/10000
4/4 [===========================

4/4 [==============================] - 0s 9ms/step - loss: 1479.1318 - val_loss: 1553.9186
Epoch 231/10000
4/4 [==============================] - 0s 9ms/step - loss: 1465.8811 - val_loss: 1540.5927
Epoch 232/10000
4/4 [==============================] - 0s 8ms/step - loss: 1452.8243 - val_loss: 1527.3354
Epoch 233/10000
4/4 [==============================] - 0s 9ms/step - loss: 1439.4702 - val_loss: 1514.2046
Epoch 234/10000
4/4 [==============================] - 0s 9ms/step - loss: 1426.7898 - val_loss: 1500.9335
Epoch 235/10000
4/4 [==============================] - 0s 9ms/step - loss: 1413.6700 - val_loss: 1487.8491
Epoch 236/10000
4/4 [==============================] - 0s 8ms/step - loss: 1400.9425 - val_loss: 1474.8923
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 1388.4353 - val_loss: 1462.0323
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 1375.6451 - val_loss: 1449.3972
Epoch 239/10000
4/4 [==============================] 

4/4 [==============================] - 0s 8ms/step - loss: 725.0056 - val_loss: 790.0256
Epoch 308/10000
4/4 [==============================] - 0s 8ms/step - loss: 718.2601 - val_loss: 783.2379
Epoch 309/10000
4/4 [==============================] - 0s 9ms/step - loss: 711.7138 - val_loss: 776.4229
Epoch 310/10000
4/4 [==============================] - 0s 9ms/step - loss: 705.0417 - val_loss: 769.6718
Epoch 311/10000
4/4 [==============================] - 0s 10ms/step - loss: 698.3790 - val_loss: 763.0067
Epoch 312/10000
4/4 [==============================] - 0s 9ms/step - loss: 691.9263 - val_loss: 756.3106
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 685.5734 - val_loss: 749.6642
Epoch 314/10000
4/4 [==============================] - 0s 10ms/step - loss: 679.0039 - val_loss: 743.2198
Epoch 315/10000
4/4 [==============================] - 0s 10ms/step - loss: 672.7672 - val_loss: 736.8123
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step 

...

Epoch 2666/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.8678 - val_loss: 92.7147
Epoch 2667/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7650 - val_loss: 93.7524
Epoch 2668/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8194 - val_loss: 93.7887
Epoch 2669/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7169 - val_loss: 90.9156
Epoch 2670/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7393 - val_loss: 90.4486
Epoch 2671/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5405 - val_loss: 90.7200
Epoch 2672/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8078 - val_loss: 90.7810
Epoch 2673/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.6463 - val_loss: 89.3448
Epoch 2674/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.6119 - val_loss: 89.3250
Epoch 2675/10000
4/4 [==============================] - 0s 7ms/step - los

4/4 [==============================] - 0s 8ms/step - loss: 5.1818 - val_loss: 93.7882
Epoch 2746/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.1784 - val_loss: 94.3881
Epoch 2747/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.3913 - val_loss: 96.1011
Epoch 2748/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.1657 - val_loss: 94.0667
Epoch 2749/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.1619 - val_loss: 91.8041
Epoch 2750/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.2877 - val_loss: 92.2411
Epoch 2751/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.1974 - val_loss: 93.5010
Epoch 2752/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.2841 - val_loss: 92.1789
Epoch 2753/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.2492 - val_loss: 91.3281
Epoch 2754/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.1423 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 4.5570 - val_loss: 92.8342
Epoch 2825/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.6364 - val_loss: 90.3456
Epoch 2826/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.6703 - val_loss: 89.9001
Epoch 2827/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5032 - val_loss: 91.8256
Epoch 2828/10000
4/4 [==============================] - 0s 8ms/step - loss: 4.6504 - val_loss: 94.4895
Epoch 2829/10000
4/4 [==============================] - 0s 8ms/step - loss: 4.6943 - val_loss: 94.3458
Epoch 2830/10000
4/4 [==============================] - 0s 8ms/step - loss: 4.6161 - val_loss: 92.0019
Epoch 2831/10000
4/4 [==============================] - 0s 8ms/step - loss: 4.5556 - val_loss: 91.2429
Epoch 2832/10000
4/4 [==============================] - 0s 8ms/step - loss: 4.5972 - val_loss: 91.8525
Epoch 2833/10000
4/4 [==============================] - 0s 8ms/step - loss: 4.5373 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 3.2213 - val_loss: 90.8385
Epoch 2904/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.2354 - val_loss: 92.2342
Epoch 2905/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.2200 - val_loss: 95.5276
Epoch 2906/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.2665 - val_loss: 96.4403
Epoch 2907/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.2663 - val_loss: 96.4250
Epoch 2908/10000
4/4 [==============================] - 0s 9ms/step - loss: 3.1763 - val_loss: 94.2960
Epoch 2909/10000
4/4 [==============================] - 0s 9ms/step - loss: 3.1482 - val_loss: 92.4543
Epoch 2910/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.2951 - val_loss: 89.6064
Epoch 2911/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.1664 - val_loss: 88.0541
Epoch 2912/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.1161 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 2.7251 - val_loss: 84.7106
Epoch 2983/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7627 - val_loss: 86.4147
Epoch 2984/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7278 - val_loss: 87.0052
Epoch 2985/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7346 - val_loss: 86.2932
Epoch 2986/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.6984 - val_loss: 86.5596
Epoch 2987/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.6668 - val_loss: 86.0698
Epoch 2988/10000
4/4 [==============================] - 0s 6ms/step - loss: 2.7088 - val_loss: 86.4691
Epoch 2989/10000
4/4 [==============================] - 0s 6ms/step - loss: 2.7334 - val_loss: 85.2075
Epoch 2990/10000
4/4 [==============================] - 0s 6ms/step - loss: 2.6700 - val_loss: 85.0290
Epoch 2991/10000
4/4 [==============================] - 0s 6ms/step - loss: 2.7600 - val_l

In [53]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [54]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,90.097031,91.698685,90.463211,91.531654,92.764458,93.437843,95.546097,95.039948,92.532639,91.429008,90.722733,93.103119,97.031815,102.113373,105.780624,107.505379,106.040565,96.683006,97.014992,94.444183,91.602745,89.827286,90.79937,94.54509,101.273834,107.510452,109.515427,109.51207,109.175491,109.156052
Target,86.678,100.174,89.777,94.81,93.446,87.54,96.706,74.086,92.676,82.865,71.18,108.826,105.694,107.106,101.893,110.551,100.587,94.22,111.377,98.614,94.022,91.546,95.674,88.927,105.688,104.711,97.53,99.651,76.71,75.456
Error,3.419029,8.475319,0.686211,3.278343,0.681541,5.897842,1.159904,20.953949,0.143364,8.564011,19.542732,15.722878,8.662186,4.99263,3.887627,3.045624,5.453568,2.463005,14.362007,4.169815,2.419258,1.718712,4.874634,5.618088,4.414169,2.799454,11.985428,9.861069,32.465492,33.70005


In [55]:
display(mae)
display(mape)

8.180598

0.094145946

In [56]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [57]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1108.3663]] - Target[1078.7640000000001]| =  Error: [[29.602295]]; MAPE:[[0.02744094]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[1173.3883]] - Target[1200.211]| =  Error: [[26.822754]]; MAPE:[[0.02234836]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[646.1433]] - Target[559.746]| =  Error: [[86.39734]]; MAPE:[[0.15435098]]


[array([[29.602295]], dtype=float32),
 array([[26.822754]], dtype=float32),
 array([[86.39734]], dtype=float32)]

47.607464

0.068046756